In [1]:
!pip install transformers datasets torch sacrebleu evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00


In [2]:
import wandb
import torch
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import warnings
from evaluate import load
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
!rm -rf ~/.cache/huggingface/transformers

In [4]:
model_name = "facebook/mbart-large-50"
model = MBartForConditionalGeneration.from_pretrained(model_name)
from transformers import MBart50Tokenizer
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="hi_IN", tgt_lang="en_XX")

train_dataset = load_dataset("opus100", "en-hi", split="train[:5%]")
val_dataset = load_dataset("opus100", "en-hi", split="validation")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/65.2M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/247k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/534319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
def preprocess_function(examples):
    inputs = [ex['hi'] for ex in examples["translation"]]
    targets = [ex['en'] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Map:   0%|          | 0/26716 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_only_model=True,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    logging_steps=1000,
    dataloader_num_workers=0
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
)

In [7]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ctarunvignesh (ctarunvignesh-). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.885500,0.306244
2,0.223800,0.282111
3,0.163900,0.286363
4,0.120500,0.304858
5,0.091000,0.332066
6,0.071600,0.356195
7,0.053600,0.377175
8,0.037100,0.422304


KeyboardInterrupt: 

In [16]:
model.save_pretrained('./hi-en-fine-tuned-v1')
tokenizer.save_pretrained('./hi-en-fine-tuned-token-v1')

('./hi-en-fine-tuned-token-v1/tokenizer_config.json',
 './hi-en-fine-tuned-token-v1/special_tokens_map.json',
 './hi-en-fine-tuned-token-v1/sentencepiece.bpe.model',
 './hi-en-fine-tuned-token-v1/added_tokens.json')

In [36]:
def translate_text(text):
    torch.manual_seed(42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).input_ids.to(device)
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translated_text

text = "यहाँ बहुत तेज़ बारिश हो रही है"
hindi = translate_text(text)
print(hindi)

There is a heavy rain here


In [26]:
!zip -r file.zip /kaggle/working/hi-en-fine-tuned-token-v1 /kaggle/working/hi-en-fine-tuned-v1
!ls
from IPython.display import FileLink
FileLink(r'file.zip')

updating: kaggle/working/hi-en-fine-tuned-token-v1/ (stored 0%)
updating: kaggle/working/hi-en-fine-tuned-token-v1/sentencepiece.bpe.model (deflated 49%)
updating: kaggle/working/hi-en-fine-tuned-token-v1/tokenizer_config.json (deflated 92%)
updating: kaggle/working/hi-en-fine-tuned-token-v1/special_tokens_map.json (deflated 61%)
updating: kaggle/working/hi-en-fine-tuned-v1/ (stored 0%)
updating: kaggle/working/hi-en-fine-tuned-v1/model.safetensors^C



zip error: Interrupted (aborting)
file.zip  hi-en-fine-tuned-token-v1  hi-en-fine-tuned-v1  logs	results  wandb


/kaggle/working/file.zip

In [30]:
test_dataset = load_dataset("opus100", "en-hi", split="test")
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

test_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print(test_results)

metric = load("sacrebleu")

def compute_metrics(pred):
  labels_ids = pred.label_ids
  pred_ids = pred.predictions

  decode_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  decode_labels = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  bleu_score = metric.compute(predictions=decode_preds, references=[[label] for label in decode_labels])
  return bleu_score

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'eval_loss': 0.42230355739593506}


In [32]:
predictions = trainer.predict(test_dataset=tokenized_test_dataset, metric_key_prefix="test")
metrics = compute_metrics(predictions)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [33]:
metrics

{'score': 18.285770341441015,
 'counts': [14519, 6251, 3247, 1789],
 'totals': [27960, 25960, 24073, 22327],
 'precisions': [51.9277539341917,
  24.07935285053929,
  13.48814024010302,
  8.01272002508174],
 'bp': 0.9537162083383937,
 'sys_len': 27960,
 'ref_len': 29285}